In [9]:
SEED=42
import os
from glob import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold, GroupKFold
import lightgbm as lgb
import warnings
from utils import util
import itertools
from imp import reload
reload(util)
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
pd.set_option('max_rows', 300)
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

In [10]:
path_lst = glob('../data/book_train.parquet/*')
stock_lst = [os.path.basename(path).split('=')[-1] for path in path_lst]
stock_lst[0], len(stock_lst)

('15', 112)

In [11]:
df_ret = util.gen_data_multi(stock_lst, data_type='train')

100%|██████████| 112/112 [21:13<00:00, 11.37s/it]


In [12]:
df_label = pd.read_csv('../data/train.csv')
train = util.gen_data_encoding(df_ret, df_label, data_type = 'train')

In [13]:
train.to_csv("../data/baseline_train.csv", index=False)

In [14]:
train

,time_id,Bnum,waproll_std5_B_mean,waproll_std5_B_std,waproll_std5_B_skew,waproll_std5_B_autocorr,waproll_std5_B_median,waproll_std10_B_mean,waproll_std10_B_std,waproll_std10_B_skew,waproll_std10_B_autocorr,waproll_std10_B_median,wapcandlef1_B,wapcandlef2_B,wapcandlef3_B,wapcandlef41_B,wapcandlef42_B,Snum,waproll_std5_S_mean,waproll_std5_S_std,waproll_std5_S_skew,waproll_std5_S_autocorr,waproll_std5_S_median,waproll_std10_S_mean,waproll_std10_S_std,waproll_std10_S_skew,waproll_std10_S_autocorr,waproll_std10_S_median,wapcandlef1_S,wapcandlef2_S,wapcandlef3_S,wapcandlef41_S,wapcandlef42_S,supBnum,waproll_std5_supB_mean,waproll_std5_supB_std,waproll_std5_supB_skew,waproll_std5_supB_autocorr,waproll_std5_supB_median,waproll_std10_supB_mean,waproll_std10_supB_std,waproll_std10_supB_skew,waproll_std10_supB_autocorr,waproll_std10_supB_median,wapcandlef1_supB,wapcandlef2_supB,wapcandlef3_supB,wapcandlef41_supB,wapcandlef42_supB,supSnum,waproll_std5_supS_mean,waproll_std5_supS_std,waproll_std5_supS_skew,waproll_std5_supS_autocorr,waproll_std5_supS_median,waproll_std10_supS_mean,waproll_std10_supS_std,waproll_std10_supS_skew,waproll_std10_supS_autocorr,waproll_std10_supS_median,wapcandlef1_supS,wapcandlef2_supS,wapcandlef3_supS,wapcandlef41_supS,wapcandlef42_supS,midBSnum,waproll_std5_midBS_mean,waproll_std5_midBS_std,waproll_std5_midBS_skew,waproll_std5_midBS_autocorr,waproll_std5_midBS_median,waproll_std10_midBS_mean,waproll_std10_midBS_std,waproll_std10_midBS_skew,waproll_std10_midBS_autocorr,waproll_std10_midBS_median,wapcandlef1_midBS,wapcandlef2_midBS,wapcandlef3_midBS,wapcandlef41_midBS,wapcandlef42_midBS,up50num,waproll_std5_up50_mean,waproll_std5_up50_std,waproll_std5_up50_skew,waproll_std5_up50_autocorr,waproll_std5_up50_median,waproll_std10_up50_mean,waproll_std10_up50_std,waproll_std10_up50_skew,waproll_std10_up50_autocorr,waproll_std10_up50_median,wapcandlef1_up50,wapcandlef2_up50,wapcandlef3_up50,wapcandlef41_up50,wapcandlef42_up50,down50num,waproll_std5_down50_mean,waproll_std5_down50_std,waproll_std5_down50_skew,waproll_std5_down50_autocorr,waproll_std5_down50_median,waproll_std10_down50_mean,waproll_std10_down50_std,waproll_std10_down50_skew,waproll_std10_down50_autocorr,waproll_std10_down50_median,wapcandlef1_down50,wapcandlef2_down50,wapcandlef3_down50,wapcandlef41_down50,wapcandlef42_down50,up25num,waproll_std5_up25_mean,waproll_std5_up25_std,waproll_std5_up25_skew,waproll_std5_up25_autocorr,waproll_std5_up25_median,waproll_std10_up25_mean,waproll_std10_up25_std,waproll_std10_up25_skew,waproll_std10_up25_autocorr,waproll_std10_up25_median,wapcandlef1_up25,wapcandlef2_up25,wapcandlef3_up25,wapcandlef41_up25,wapcandlef42_up25,down25num,waproll_std5_down25_mean,waproll_std5_down25_std,waproll_std5_down25_skew,waproll_std5_down25_autocorr,waproll_std5_down25_median,waproll_std10_down25_mean,waproll_std10_down25_std,waproll_std10_down25_skew,waproll_std10_down25_autocorr,waproll_std10_down25_median,wapcandlef1_down25,wapcandlef2_down25,wapcandlef3_down25,wapcandlef41_down25,wapcandlef42_down25,up75num,waproll_std5_up75_mean,waproll_std5_up75_std,waproll_std5_up75_skew,waproll_std5_up75_autocorr,waproll_std5_up75_median,waproll_std10_up75_mean,waproll_std10_up75_std,waproll_std10_up75_skew,waproll_std10_up75_autocorr,waproll_std10_up75_median,wapcandlef1_up75,wapcandlef2_up75,wapcandlef3_up75,wapcandlef41_up75,wapcandlef42_up75,down75num,waproll_std5_down75_mean,waproll_std5_down75_std,waproll_std5_down75_skew,waproll_std5_down75_autocorr,waproll_std5_down75_median,waproll_std10_down75_mean,waproll_std10_down75_std,waproll_std10_down75_skew,waproll_std10_down75_autocorr,waproll_std10_down75_median,wapcandlef1_down75,wapcandlef2_down75,wapcandlef3_down75,wapcandlef41_down75,wapcandlef42_down75,stock_id,book_wap1_lambda_,book_wap2_lambda_,book_wap_mean_lambda_,book_wap_diff_lambda_,book_price_spread_lambda_,book_bid_spread_lambda_,book_ask_spread_lambda_,book_total_volume_lambda_,book_volume_imbalance_lambda_,pricesum,priceme

In [24]:
train["stock_id"] = train["stock_id"].astype("category")
train = train.sample(frac=1, random_state=SEED).reset_index(drop=True)

In [25]:
feature_cols = [c for c in train.columns if c not in ['target', 'time_id']]
target_col = 'target'

In [ ]:
params = {
    'random_state': SEED,
    'objective': 'rmse',  
    'boosting_type': 'gbdt',
#     'boosting_type': 'dart',
#     'num_leaves': 100,
#     'learning_rate': 0.1,
#     'colsample_bytree': 0.8,
#     'subsample': 0.8,
    'n_estimators': 10000,
    'min_child_samples': int(0.01*train.shape[0])
}
params = {
    'n_estimators': 20000,
    'objective': 'rmse', # poisson
    'boosting_type': 'gbdt', # dart
    'max_depth': -1,
    'learning_rate': 0.01,
    'subsample': 0.8,
    'subsample_freq': 4,
    'feature_fraction': 0.8,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'seed':SEED,
    'early_stopping_rounds': 500,
    'verbose': -1
}
oof_predictions = np.zeros(train.shape[0])
kfold = GroupKFold(n_splits=5).split(train, groups=train.time_id)
for fold, (trn_ind, val_ind) in enumerate(kfold):
    x_train, x_val = train.loc[trn_ind, feature_cols], train.loc[val_ind, feature_cols]
    y_train, y_val = train.loc[trn_ind, target_col], train.loc[val_ind, target_col]

    train_weights = 1 / np.square(y_train)
    val_weights = 1 / np.square(y_val)

    # Fit with sklearn API
    model = lgb.LGBMRegressor(**params)
    model.fit(x_train, 
              y_train, 
              sample_weight=train_weights,
              eval_set=[(x_val, y_val)],
              eval_sample_weight=[val_weights],
              eval_metric='rmse',
              early_stopping_rounds=100,
              verbose=100)

    # Add predictions to the out of folds array
    oof_predictions[val_ind] = model.predict(x_val)
oof_score = rmspe(train['target'], oof_predictions)
print(oof_score) # 0.228

[LightGBM] [Warning] early_stopping_round is set=500, early_stopping_rounds=500 will be ignored. Current value: early_stopping_round=500
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
Training until validation scores don't improve for 500 rounds
[100]	valid_0's rmse: 0.000672011
[200]	valid_0's rmse: 0.00054678
[300]	valid_0's rmse: 0.000519049
[400]	valid_0's rmse: 0.000511332
[500]	valid_0's rmse: 0.000508019
[600]	valid_0's rmse: 0.000505722
[700]	valid_0's rmse: 0.000504022
[800]	valid_0's rmse: 0.000502838
[900]	valid_0's rmse: 0.000502067
[1000]	valid_0's rmse: 0.000501523
[1100]	valid_0's rmse: 0.000500707
[1200]	valid_0's rmse: 0.000500293
[1300]	valid_0's rmse: 0.000499801
[1400]	valid_0's rmse: 0.00049952